# Load data

In [1]:
import glob
import os

In [2]:
!unzip /content/oreilly-open-source-llm-master.zip

Archive:  /content/oreilly-open-source-llm-master.zip
bc468428a17a2e1dfc43a9bb7aba63077897d411
   creating: oreilly-open-source-llm-master/
   creating: oreilly-open-source-llm-master/Week 1/
  inflating: oreilly-open-source-llm-master/Week 1/01-similarity-embeddings.ipynb  
  inflating: oreilly-open-source-llm-master/Week 1/02-similarity-cross-encoder.ipynb  
  inflating: oreilly-open-source-llm-master/Week 1/03-similarity-langchain.ipynb  
  inflating: oreilly-open-source-llm-master/Week 1/04-txtai.ipynb  
  inflating: oreilly-open-source-llm-master/Week 1/README.md  
   creating: oreilly-open-source-llm-master/Week 1/un/
   creating: oreilly-open-source-llm-master/Week 1/un/TXT/
   creating: oreilly-open-source-llm-master/Week 1/un/TXT/Session 77 - 2022/
  inflating: oreilly-open-source-llm-master/Week 1/un/TXT/Session 77 - 2022/AGO_77_2022.txt  
  inflating: oreilly-open-source-llm-master/Week 1/un/TXT/Session 77 - 2022/ALB_77_2022.txt  
  inflating: oreilly-open-source-llm-master/

In [3]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


In [4]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.text import TextLoader
loader = DirectoryLoader("/content/oreilly-open-source-llm-master/Week 1/un/TXT/Session 77 - 2022/", glob="**/*.txt", loader_cls=TextLoader)
data = loader.load()

In [6]:
len(data)

193

In [7]:
data[0]

Document(page_content='On behalf of the Government and the people of Nigeria, I congratulate the President on his well-deserved election to lead the General Assembly at its seventy-seventh session and assure him of the full support and cooperation of the Nigerian delegation during his tenure. I commend his predecessor, His Excellency Mr. Abdulla Shahid, for the Assembly’s many remarkable achievements under his leadership during these challenging times. I would also like to congratulate the Secretary-General, Mr. Antonio Guterres, on his ceaseless and untiring efforts to promote peace, security and development, which are very much in line with his exalted role.\nThe first time I might have been able to address the Assembly was in 1984, when I was the Military Head of State of the Federal Republic of Nigeria. Some 31 years later, in 2016, I had the great privilege to personally address the Assembly as the democratically elected President of my country (see A/71/PV.4 A). As I approach the

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

In [11]:
#!pip install sentence-transformers

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.vectorstores.sklearn import SKLearnVectorStore

vectorstore = SKLearnVectorStore.from_documents(documents=all_splits,
                                                embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

In [13]:
question = "Are poorer countries more affected by the climate crisis?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [14]:
docs

[Document(page_content='The effects of climate change are uneven and reflect a historical injustice whereby those who are least responsible suffer the most. The Philippines, for example, is a net carbon sink. We absorb more carbon dioxide than we emit. Yet we are the fourth most vulnerable country to the effects of climate change.\nThat injustice must be corrected and those who need to do more must act now. We accept our share of responsibility and will continue to do our part to avert that collective disaster.', metadata={'id': '01f74d83-ad51-4c52-afac-215d0143ab18', 'source': '/content/oreilly-open-source-llm-master/Week 1/un/TXT/Session 77 - 2022/PHL_77_2022.txt'}),
 Document(page_content='Although climate change and the resulting extreme weather conditions occur throughout the globe, the crisis largely affects the minimally resilient and those least responsible for causing the problem. A case in point is our region, the Horn of Africa, which is badly hit by record-level drought in 

In [15]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)
df = pd.DataFrame([{"source": d.metadata["source"], "text": d.page_content} for d in docs])
df

,source,text
0,/content/oreilly-open-source-llm-master/Week 1/un/TXT/Session 77 - 2022/PHL_77_2022.txt,"The effects of climate change are uneven and reflect a historical injustice whereby those who are least responsible suffer the most. The Philippines, for example, is a net carbon sink. We absorb more carbon dioxide than we emit. Yet we are the fourth most vulnerable country to the effects of climate change.\nThat injustice must be corrected and those who need to do more must act now. We accept our share of responsibility and will continue to do our part to avert that collective disaster."
1,/content/oreilly-open-source-llm-master/Week 1/un/TXT/Session 77 - 2022/ETH_77_2022.txt,"Although climate change and the resulting extreme weather conditions occur throughout the globe, the crisis largely affects the minimally resilient and those least responsible for causing the problem. A case in point is our region, the Horn of Africa, which is badly hit by record-level drought in some parts, while sustaining extreme flooding in other parts."
2,/content/oreilly-open-source-llm-master/Week 1/un/TXT/Session 77 - 2022/CPV_77_2022.txt,"example, my country, Cabo Verde, in the past 15 years — between 2007 and 2022 — has suffered the economic and social impact of multiple crises: the economic and financial crisis of 2007-2008, at the very moment when we graduated from the list of least developed countries; the coronavirus disease pandemic, which caused a recession of 14.6 per cent in 2020; the ongoing inflationary impact of world events; and, in the last five years, one of the most profound and most serious droughts in the"
3,/content/oreilly-open-source-llm-master/Week 1/un/TXT/Session 77 - 2022/BTN_77_2022.txt,most vulnerable to adapt to the changing climate and deal with its consequences.
